# Jupyter notebook sample

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
# pip install python-dotenv

import xml.etree.ElementTree as ET
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the secrets
app_token = os.getenv("APP_TOKEN")
username = os.getenv("OPEN_DATA_NYC_USERNAME")
password = os.getenv("OPEN_DATA_NYC_PASSWORD")

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofnewyork.us",
                 app_token,
                  username=username,
                  password=password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("pitm-atqc", limit=1000)

# Convert to pandas DataFrame
df_restaurants = pd.DataFrame.from_records(results)

In [4]:
df_restaurants.head()

,objectid,globalid,seating_interest_sidewalk,restaurant_name,legal_business_name,doing_business_as_dba,bulding_number,street,borough,zip,...,community_board,council_district,census_tract,bin,bbl,nta,roadway_dimensions_length,roadway_dimensions_width,roadway_dimensions_area,landmarkdistrict_terms
0,100,c4b3155b-31a0-4e95-846f-fce09f245437,sidewalk,Pomp and Circumstance Hospitality,Pomp and Circumstance Hospitality LLC,Pomp and Circumstance Hospitality LLC,577,Lorimer Street,Brooklyn,11211,...,1,34,501,3068653,3027560028,East Williamsburg,NaN,NaN,NaN,NaN
1,1000,753495d8-4429-43e5-85a3-dcf6230ef749,both,Charm Kao,193 Schemerhorn INC,Charm Kao,193,Schermerhorn St.,Brooklyn,11201,...,2,33,37,3000493,3001640041,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,24,8,192,NaN
2,10000,{3842B5C5-EF04-41A4-8216-D6EA627DCE5E},openstreets,SAKE BAR HAGI 46,"HAMA NEW YORK, INC.",SAKE BAR HAGI 46,358,W. 46TH STREET,Manhattan,10036,...,4,3,121,1025025,1010360057,Clinton,NaN,NaN,NaN,NaN
3,10001,{C212A0FC-C115-4425-8F95-931B12C5F86A},openstreets,Yum yum too,Boythaicorp,Boythaicorp,662,9ave,Manhattan,10036,...,4,3,127,1025038,1010370001,Clinton,NaN,NaN,NaN,NaN
4,10002,{DA48265D-7730-416F-8E1C-EBC8C8ACE2C2},openstreets,Xochil Pizza Corp,Xochil Pizza Corp,Xochil Pizza Corp,4632,5th Avenue,Brooklyn,11220,...,7,38,80,NaN,NaN,Sunset Park West,NaN,NaN,NaN,NaN


In [5]:
# Helper function to dynamically handle missing cells
def extract_row_data(cells, expected_columns):
    """Extract row data and ensure it matches the number of headers."""
    row_data = []
    for i in range(expected_columns):
        try:
            cell = cells[i].find(".//ss:Data", ns)
            row_data.append(cell.text if cell is not None else None)
        except IndexError:
            row_data.append(None)  # Append None if the column is missing
    return row_data

# Parse the XML file
file_path = './datasets/Interactive_Map_Data.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# Namespace dictionary for handling XML namespaces
ns = {'ss': 'urn:schemas-microsoft-com:office:spreadsheet'}

# Retrieve all worksheets
worksheets = root.findall(".//ss:Worksheet", ns)

# Dictionary to store DataFrames for each sheet
dataframes = {}

# Specify header row index for each sheet
header_row_mapping = {
    'Full Map List': 1,
    'Notable Directors': 0,
    'Items without location': 2
}

# Loop through each worksheet
for sheet in worksheets:
    sheet_name = sheet.attrib.get(f"{{{ns['ss']}}}Name")  # Get the sheet name
    rows = sheet.findall(".//ss:Row", ns)  # Find all rows in the sheet

    if not rows:
        continue  # Skip empty sheets

    # Determine the header row index (default to 0 if not specified)
    header_row_index = header_row_mapping.get(sheet_name, 0)
    if header_row_index >= len(rows):
        print(f"Invalid header row index for sheet: {sheet_name}. Skipping...")
        continue

    # Extract headers from the specified row
    header_row = rows[header_row_index]
    headers = []
    for cell in header_row.findall(".//ss:Cell", ns):
        data = cell.find(".//ss:Data", ns)
        headers.append(data.text if data is not None else None)
    expected_columns = len(headers)

    # Extract data (skip up to the header row)
    data = []
    for row in rows[header_row_index + 1:]:  # Start after the header row
        cells = row.findall(".//ss:Cell", ns)
        row_data = extract_row_data(cells, expected_columns)
        data.append(row_data)

    # Create a DataFrame for the sheet
    df = pd.DataFrame(data, columns=headers)
    dataframes[sheet_name] = df  # Store in the dictionary

    print(f"Processed sheet: {sheet_name} with {len(df)} rows")

# Example: Access a specific sheet
print("\nSample from 'Items without location':")
if 'Items without location' in dataframes:
    print(dataframes['Items without location'].head())

# Save each DataFrame to a separate CSV file
for sheet_name, df in dataframes.items():
    #df.to_csv(f"./outputs/{sheet_name.replace(' ', '_')}.csv", index=False)
    df.head()


Processed sheet: Full Map List with 233 rows
Processed sheet: Notable Directors with 21 rows
Processed sheet: Items without location with 7 rows

Sample from 'Items without location':
                                     Film       Year      file Name  \
0  Video: Rolling Stones "Love is Strong"       1994           None   
1                     Commercial for Nike       None           None   
2       Commercial for Tony Awards & Visa       2001           None   
3                         Naked City (TV)  1958-1963  nakedcity1_pf   
4                         Naked City (TV)  1958-1963  nakedcity2_pf   

           Agency Credit             Director/Filmmaker Name  \
0                   None                                None   
1                   None                                None   
2                   None                                None   
3  Courtesy of Photofest  John Brahm\nJules Bricken\n (more)   
4  Courtesy of Photofest  John Brahm\nJules Bricken\n (more)   

    